# Define Business Vocabulary


## Authorisation

In [2]:
import json
import requests # type: ignore
import time

class credentials :

    file_credentials = ""
    url_server = "https://cpd-cpd.apps.6645c6d6ca5b92001e29286f.cloud.techzone.ibm.com"
    username = "admin"
    apikey = "SfpLD0yMQFh4xpdOrgPuTK9AdBtEVEqF1gK2HSlw"
    access_token = ""

    def __init__(self, file_credentials):

        if file_credentials != "" :
            try :
                with open(file_credentials) as f :
                    data = json.load(f)
                    self.url_server = data["url_server"]
                    self.username = data["username"]
                    self.apikey = data["api_key"]
                    self.file_credentials = file_credentials
            except :
                print("Error with the file ", file_credentials)
    
   
    def urlRequest(self, urlSuffix):
        return self.url_server + urlSuffix

    def get_bearer_token(self):
        
        # Get a bearer token with the API key - Cloud Pak for Data SaaS
        # url = "https://iam.cloud.ibm.com/identity/token"
        # headers = {"Content-Type" : "application/x-www-form-urlencoded"}
        # data = "grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey=" + apikey
        # r = requests.post(url, headers=headers, data=data)
        # access_token = r.json()["access_token"]

        # Get a bearer token with the API key - Cloud Pak for Data Software
        urlSuffix = "/icp4d-api/v1/authorize"
        headers = {'Accept': 'application/json', 'Content-type': 'application/json'}
        data = {"username" : self.username, "api_key" : self.apikey}
        r = requests.post(self.urlRequest(urlSuffix), headers=headers, data=json.dumps(data))

        if r.status_code != 200:
            print("Error with the request. Code: ", r.status_code)
            print(r.text)
        else :
            try:
                self.access_token = r.json()["token"]
            except KeyError:
                print("Error with the token. Code: ", r.status_code)
                print("Hint: check the credentials file ", self.file_credentials)
                print(r.text)
                
            return self.access_token
    
# FILE_CREDENTIALS = "../../python/ikcapikey.json"
FILE_CREDENTIALS = ""

myconn = credentials(FILE_CREDENTIALS)
access_token = myconn.get_bearer_token()


## Define the Business Vocabulary


### 1. Create Categories
   

In [3]:
print("---- Import Categories from CSV----")

IMPORT_CSV_FILE = "../../artifacts/governance-categories.csv"
urlSuffix='/v3/governance_artifact_types/category/import?merge_option=all'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
files = {'file': (IMPORT_CSV_FILE, open(IMPORT_CSV_FILE, 'rb'), 'text/csv')}

r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, files=files)

if r.status_code == 200 :
    status = r.json()["status"]
    print("Import finished. Status = ", status)
    print(r.text)
elif r.status_code == 202 :
    process_id = r.json()["process_id"]
    print(f"----- Import process started: {process_id} ----- ")
    print("----- Entering wait loop ------")
    urlSuffix='/v3/governance_artifact_types/import/status/' + process_id
    headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
    while True :
        r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)
        if r.status_code != 200 :
            print("Error with the request. Code: ", r.status_code)
            print(r.text)
        status = r.json()["status"]
        if status != "IN_PROGRESS" :
            break
        else :
            print ("Import in progess, please wait")
            time.sleep(5)
else :
    print("Error with the request. Code: ", r.status_code)
    print(r.text)


---- Import Categories from CSV----


FileNotFoundError: [Errno 2] No such file or directory: '.../../artifacts/governance-categories.csv'

### 2. Update Classifications

#### 2.a. Change the definitions 

In [5]:
print("---- Update Classifications from CSV----")

IMPORT_CSV_FILE = "../governance-classifications.csv" 
urlSuffix='/v3/governance_artifact_types/classification/import?merge_option=specified'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
files = {'file': (IMPORT_CSV_FILE, open(IMPORT_CSV_FILE, 'rb'), 'text/csv')}

r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, files=files)

if r.status_code == 200 :
    status = r.json()["status"]
    print("Import finished. Status = ", status)
    print(r.text)

elif r.status_code == 202 :
    process_id = r.json()["process_id"]
    print(f"----- Import process started: {process_id} ----- ")
    print("----- Entering wait loop ------")
    urlSuffix='/v3/governance_artifact_types/import/status/' + process_id
    headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}

    while True :
        r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)
        if r.status_code != 200 :
            print("Error with the request. Code: ", r.status_code)
            print(r.text)
            break
        status = r.json()["status"]
        if status != "IN_PROGRESS" :
            break
        else :
            print ("Import in progess, please wait")
            time.sleep(5)
else :
    print("Error with the request. Code: ", r.status_code)
    print(r.text)

workflow_id = r.json()["workflow_id"]


---- Update Classifications from CSV----
Import finished. Status =  SUCCEEDED
{
  "process_id": "4ea91cdb-6c88-47ec-80aa-eff1c1083d23",
  "status": "SUCCEEDED",
  "step_number": 16,
  "total_steps": 16,
  "step_message": "Artifacts import finished",
  "creation_time": "2024-05-21T08:40:15.150Z",
  "completion_time": "2024-05-21T08:40:25.870Z",
  "tenant_id": "999",
  "workflow_id": "c063eec0-174d-11ef-8e75-0a580a820497",
  "heartbeat_time": "2024-05-21T08:40:25.870Z",
  "messages": {
    "resources": [],
    "offset": 0,
    "set_uri": false,
    "limit": 200,
    "count": 0
  },
  "operations_count": {
    "relationship": {
      "IMPORT_CREATE": 8
    },
    "classification": {
      "IMPORT_CREATE": 0,
      "IMPORT_MODIFY": 4
    }
  }
}


#### 2.b. Publish the definitions

Before executing this cell, you may want to check the "Task Inbox" in CloudPak for Data if you are not sure about what will be published

In [6]:
urlSuffix='/v3/workflows/' + workflow_id + '?include_user_tasks=true'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)

user_tasks = r.json()["entity"]["user_tasks"]
for i in user_tasks :
    if i["metadata"]["workflow_id"] == workflow_id :
        task_id = i["metadata"]["task_id"]

urlSuffix='/v3/workflow_user_tasks/' + task_id + '/actions'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
payload = {'action': 'complete', 'form_properties': [{'id': 'action', 'value': '#publish'}]}
r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, json=payload)

if r.status_code == 202 or r.status_code == 204 :
    print("Publish Successful, Code = ", r.status_code)
else :
    print("Error in publishing artifacts, Code = ", r.status_code)
    print(r.text)

Publish Successful, Code =  204


### 3. Create Data Classes

#### 3.a. Add new Data Classes

In [7]:
print("---- Create Data Classes from CSV----")

IMPORT_CSV_FILE = "../governance-data-classes.csv" 
urlSuffix='/v3/governance_artifact_types/data_class/import?merge_option=all'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
files = {'file': (IMPORT_CSV_FILE, open(IMPORT_CSV_FILE, 'rb'), 'text/csv')}

r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, files=files)

if r.status_code == 200 :
    status = r.json()["status"]
    print("Import finished. Status = ", status)
    print(r.text)

elif r.status_code == 202 :
    process_id = r.json()["process_id"]
    print(f"----- Import process started: {process_id} ----- ")
    print("----- Entering wait loop ------")
    urlSuffix='/v3/governance_artifact_types/import/status/' + process_id
    headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}

    while True :
        r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)
        if r.status_code != 200 :
            print("Error with the request. Code: ", r.status_code)
            print(r.text)
            break
        status = r.json()["status"]
        if status != "IN_PROGRESS" :
            break
        else :
            print ("Import in progess, please wait")
            time.sleep(5)
else :
    print("Error with the request. Code: ", r.status_code)
    print(r.text)

workflow_id = r.json()["workflow_id"]

---- Create Data Classes from CSV----
Import finished. Status =  SUCCEEDED
{
  "process_id": "4bf54fb3-a26f-45f8-9944-7840e6975852",
  "status": "SUCCEEDED",
  "step_number": 16,
  "total_steps": 16,
  "step_message": "Artifacts import finished",
  "creation_time": "2024-05-21T08:42:08.457Z",
  "completion_time": "2024-05-21T08:42:16.680Z",
  "tenant_id": "999",
  "workflow_id": "0444f2e7-174e-11ef-8e75-0a580a820497",
  "heartbeat_time": "2024-05-21T08:42:16.680Z",
  "messages": {
    "resources": [],
    "offset": 0,
    "set_uri": false,
    "limit": 200,
    "count": 0
  },
  "operations_count": {
    "data_class": {
      "IMPORT_CREATE": 15,
      "IMPORT_MODIFY": 0
    },
    "relationship": {
      "IMPORT_CREATE": 15
    }
  }
}


#### 3.b. Publish the changes

In [8]:
urlSuffix='/v3/workflows/' + workflow_id + '?include_user_tasks=true'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)

user_tasks = r.json()["entity"]["user_tasks"]
for i in user_tasks :
    if i["metadata"]["workflow_id"] == workflow_id :
        task_id = i["metadata"]["task_id"]

urlSuffix='/v3/workflow_user_tasks/' + task_id + '/actions'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
payload = {'action': 'complete', 'form_properties': [{'id': 'action', 'value': '#publish'}]}
r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, json=payload)

if r.status_code == 202 or r.status_code == 204 :
    print("Publish Successful, Code = ", r.status_code)
else :
    print("Error in publishing artifacts, Code = ", r.status_code)
    print(r.text)


Publish Successful, Code =  204


### 4. Create Business Terms

#### 4.a. Add new Business Terms

In [9]:
print("---- Create Business Terms from CSV----")

IMPORT_CSV_FILE = "../governance-business-terms.csv"
urlSuffix='/v3/governance_artifact_types/glossary_term/import?merge_option=all'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
files = {'file': (IMPORT_CSV_FILE, open(IMPORT_CSV_FILE, 'rb'), 'text/csv')}

r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, files=files)

if r.status_code == 200 :
    status = r.json()["status"]
    print("Import finished. Status = ", status)

elif r.status_code == 202 :
    process_id = r.json()["process_id"]
    print(f"----- Import process started: {process_id} ----- ")
    print("----- Entering wait loop ------")
    urlSuffix='/v3/governance_artifact_types/import/status/' + process_id
    headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}

    while True :
        r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)
        if r.status_code != 200 :
            print("Error with the request. Code: ", r.status_code)
            print(r.text)
            break
        status = r.json()["status"]
        if status != "IN_PROGRESS" :
            break
        else :
            print ("Import in progess, please wait")
            time.sleep(5)
else :
    print("Error with the request. Code: ", r.status_code)
    print(r.text)

workflow_id = r.json()["workflow_id"]

---- Create Business Terms from CSV----
Import finished. Status =  SUCCEEDED


#### 4.b. Publish the changes


In [10]:
urlSuffix='/v3/workflows/' + workflow_id + '?include_user_tasks=true'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)

user_tasks = r.json()["entity"]["user_tasks"]
for i in user_tasks :
    if i["metadata"]["workflow_id"] == workflow_id :
        task_id = i["metadata"]["task_id"]

urlSuffix='/v3/workflow_user_tasks/' + task_id + '/actions'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
payload = {'action': 'complete', 'form_properties': [{'id': 'action', 'value': '#publish'}]}
r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, json=payload)

if r.status_code == 202 or r.status_code == 204 :
    print("Publish Successful, Code = ", r.status_code)
else :
    print("Error in publishing artifacts, Code = ", r.status_code)
    print(r.text)


Publish Successful, Code =  202


### 5. Create Reference Data

#### 5.a Add new Reference Data

In [11]:
print("---- Create Reference Data from CSV----")

IMPORT_CSV_FILE = "../governance-reference-data.csv"
urlSuffix='/v3/governance_artifact_types/reference_data/import?merge_option=all'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
files = {'file': (IMPORT_CSV_FILE, open(IMPORT_CSV_FILE, 'rb'), 'text/csv')}

r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, files=files)

if r.status_code == 200 :
    status = r.json()["status"]
    print("Import finished. Status = ", status)

elif r.status_code == 202 :
    process_id = r.json()["process_id"]
    print(f"----- Import process started: {process_id} ----- ")
    print("----- Entering wait loop ------")
    urlSuffix='/v3/governance_artifact_types/import/status/' + process_id
    headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}

    while True :
        r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)
        if r.status_code != 200 :
            print("Error with the request. Code: ", r.status_code)
            print(r.text)
            break
        status = r.json()["status"]
        if status != "IN_PROGRESS" :
            break
        else :
            print ("Import in progess, please wait")
            time.sleep(5)
else :
    print("Error with the request. Code: ", r.status_code)
    print(r.text)

workflow_id = r.json()["workflow_id"]

---- Create Reference Data from CSV----
Import finished. Status =  SUCCEEDED


#### 5.b Publish the changes


In [12]:
urlSuffix='/v3/workflows/' + workflow_id + '?include_user_tasks=true'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)

user_tasks = r.json()["entity"]["user_tasks"]
for i in user_tasks :
    if i["metadata"]["workflow_id"] == workflow_id :
        task_id = i["metadata"]["task_id"]

urlSuffix='/v3/workflow_user_tasks/' + task_id + '/actions'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
payload = {'action': 'complete', 'form_properties': [{'id': 'action', 'value': '#publish'}]}
r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, json=payload)

if r.status_code == 202 or r.status_code == 204 :
    print("Publish Successful, Code = ", r.status_code)
else :
    print("Error in publishing artifacts, Code = ", r.status_code)
    print(r.text)

Publish Successful, Code =  202


### 6. Load Department Lookup Data


#### 6.a Add the lookup data

In [13]:
print("---- Load Department Lookup Data from CSV----")

IMPORT_CSV_FILE = "../governance-reference-department.csv"

artifact_id = None
version_id = None
urlSuffix='/v3/governance_artifact_types/reference_data?workflow_status=published&limit=200'
headers = {"accept": "application/json" ,"Authorization" : "Bearer " + access_token}

r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)

if r.status_code == 200 :
    for i in r.json()["resources"] :
        if i["name"] == "Department Lookup" :
            artifact_id = i["artifact_id"]
            version_id = i["version_id"]
            print("artifact_id = ", artifact_id, " version_id = " , version_id)
            break
else :
    print("Error in retrieving reference data artifacts, Code = ", r.status_code)
    print(r.text)

if artifact_id is None or version_id is None:
    print("Department Lookup not found")
else :    
    urlSuffix='/v4/reference_data_sets/' + artifact_id + '/versions/' + version_id + '/value_imports'
    headers = {"Authorization" : "Bearer " + access_token }
    import_parameters = {
        "artifact_id_mode": False,
        "code": "DEPARTMENT_CODE",
        "first_row_header": True,
        "import_relationships_only": False,
        "skip_workflow_if_possible": False, 
        "trim_white_spaces": True,
        "value": "DEPARTMENT_EN",
        "value_conflicts": "OVERWRITE" 
    }
    files={
        'import_csv_file'   : ('import_csv_file', open(IMPORT_CSV_FILE,'rb') ),
        'import_parameters' : (None, str(import_parameters))   
    }
    
    r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, files=files)

    if r.status_code == 202 :
        import_id = r.json()["import_info"]["import_id"]
        print(f"----- Import process started: {import_id} ----- ")
        print("----- Entering wait loop ------")
        urlSuffix='/v4/reference_data_sets/' + artifact_id + '/versions/' + version_id + '/value_imports/' + import_id
        headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}

        workflow_id = r.json()["workflow_id"]

        while True :
            r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)
            if r.status_code != 200 :
                print("Error with the request. Code: ", r.status_code)
                print(r.text)
                break
            status = r.json()["import_info"]["import_state"]
            if status != "IN_PROGRESS" :
                break
            else :
                print ("Import in progess, please wait")
                time.sleep(5)
        print("Import finished. Status = ", r.status_code)
    else :
        print("Error with the request. Code: ", r.status_code)
        print(r.text)

---- Load Department Lookup Data from CSV----
artifact_id =  cfd84306-3f31-4f65-8336-4a9639d05f5a  version_id =  f4ad0be7-e41e-4dbb-8c7f-59f9ef22c8b4_0
----- Import process started: 12813349-a2cf-4011-b233-aa13575def27 ----- 
----- Entering wait loop ------
Import finished. Status =  200


#### 6.b. Publish the changes

In [14]:
urlSuffix='/v3/workflows/' + workflow_id + '?include_user_tasks=true'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)

user_tasks = r.json()["entity"]["user_tasks"]
for i in user_tasks :
    if i["metadata"]["workflow_id"] == workflow_id :
        task_id = i["metadata"]["task_id"]

urlSuffix='/v3/workflow_user_tasks/' + task_id + '/actions'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
payload = {'action': 'complete', 'form_properties': [{'id': 'action', 'value': '#publish'}]}
r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, json=payload)

if r.status_code == 202 or r.status_code == 204 :
    print("Publish Successful, Code = ", r.status_code)
else :
    print("Error in publishing artifacts, Code = ", r.status_code)
    print(r.text)

Publish Successful, Code =  202


### 7. Load Position Lookup Data

#### 7.a. Add the new data

In [15]:

print("---- Load Position Lookup Data from CSV----")

IMPORT_CSV_FILE = "../governance-reference-position.csv"

artifact_id = None
version_id = None
urlSuffix='/v3/governance_artifact_types/reference_data?workflow_status=published&limit=200'
headers = {"accept": "application/json" ,"Authorization" : "Bearer " + access_token}

r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)

if r.status_code == 200 :
    for i in r.json()["resources"] :
        if i["name"] == "Position Lookup" :
            artifact_id = i["artifact_id"]
            version_id = i["version_id"]
            print("artifact_id = ", artifact_id, " version_id = " , version_id)
            break
else :
    print("Error in retrieving reference data artifacts, Code = ", r.status_code)
    print(r.text)

if artifact_id is None or version_id is None:
    print("Position Lookup not found")
else :    
    urlSuffix='/v4/reference_data_sets/' + artifact_id + '/versions/' + version_id + '/value_imports'
    headers = {"Authorization" : "Bearer " + access_token }
    import_parameters = {
        "artifact_id_mode": False,
        "code": "POSITION_CODE",
        "first_row_header": True,
        "import_relationships_only": False,
        "skip_workflow_if_possible": False, 
        "trim_white_spaces": True,
        "value": "POSITION_EN",
        "value_conflicts": "OVERWRITE" 
    }
    files={
        'import_csv_file'   : ('import_csv_file', open(IMPORT_CSV_FILE,'rb') ),
        'import_parameters' : (None, str(import_parameters))   
    }
    
    r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, files=files)

    if r.status_code == 202 :
        import_id = r.json()["import_info"]["import_id"]
        print(f"----- Import process started: {import_id} ----- ")
        print("----- Entering wait loop ------")
        urlSuffix='/v4/reference_data_sets/' + artifact_id + '/versions/' + version_id + '/value_imports/' + import_id
        headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}

        workflow_id = r.json()["workflow_id"]

        while True :
            r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)
            if r.status_code != 200 :
                print("Error with the request. Code: ", r.status_code)
                print(r.text)
                break
            status = r.json()["import_info"]["import_state"]
            if status != "IN_PROGRESS" :
                break
            else :
                print ("Import in progess, please wait")
                time.sleep(5)
        print("Import finished. Status = ", r.status_code)
    else :
        print("Error with the request. Code: ", r.status_code)
        print(r.text)

---- Load Position Lookup Data from CSV----
artifact_id =  26051dfb-c580-4a74-87d4-4b605525a61b  version_id =  43c25345-8ecf-4bb8-981c-9b442ecb618a_0
----- Import process started: 98a08973-af4b-42e2-9eec-a9c87768f256 ----- 
----- Entering wait loop ------
Import finished. Status =  200


#### 7.b. Publish the draft

In [16]:
urlSuffix='/v3/workflows/' + workflow_id + '?include_user_tasks=true'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
r = requests.get(myconn.urlRequest(urlSuffix), headers=headers)

user_tasks = r.json()["entity"]["user_tasks"]
for i in user_tasks :
    if i["metadata"]["workflow_id"] == workflow_id :
        task_id = i["metadata"]["task_id"]

urlSuffix='/v3/workflow_user_tasks/' + task_id + '/actions'
headers = {"accept": "application/json", "Authorization" : "Bearer " + access_token}
payload = {'action': 'complete', 'form_properties': [{'id': 'action', 'value': '#publish'}]}
r = requests.post(myconn.urlRequest(urlSuffix), headers=headers, json=payload)

if r.status_code == 202 or r.status_code == 204 :
    print("Publish Successful, Code = ", r.status_code)
else :
    print("Error in publishing artifacts, Code = ", r.status_code)
    print(r.text)

Publish Successful, Code =  202
